In [1]:
import pandas as pd
import numpy as np

In [2]:
data_df = pd.read_excel('cleaned_project_data_with_locations.xlsx')
data_df.head()

,Index,RYear,RMonth,Location,Bedrooms,Zip,State,Date_Start,Date_End,Days,UnitOfM,UtilType,MajorType,Units,Cost,Row #,Date Code
0,6007,2023,1,Harbor Pointe Apartments,311,30263,GA,2023-01-13,2023-02-13,31,kWh,Electric,Electric,10768.0,1082.74,6007,2023.01
1,6008,2023,2,Harbor Pointe Apartments,311,30263,GA,2023-02-13,2023-03-16,31,kWh,Electric,Electric,5433.0,844.22,6008,2023.02
2,6009,2023,3,Harbor Pointe Apartments,311,30263,GA,2023-03-16,2023-04-14,29,kWh,Electric,Electric,5052.0,826.21,6009,2023.03
3,6010,2023,4,Harbor Pointe Apartments,311,30263,GA,2023-04-14,2023-05-14,30,kWh,Electric,Electric,3973.0,775.16,6010,2023.04
4,6011,2023,1,Harbor Pointe Apartments,311,30263,GA,2023-01-12,2023-02-13,32,kWh,Electric,Electric,27160.0,2550.38,6011,2023.01


In [3]:
# use per day has to be calculated before grouping
data_df["Use per Day"] = data_df["Units"] / data_df["Days"]
data_df.head()

,Index,RYear,RMonth,Location,Bedrooms,Zip,State,Date_Start,Date_End,Days,UnitOfM,UtilType,MajorType,Units,Cost,Row #,Date Code,Use per Day
0,6007,2023,1,Harbor Pointe Apartments,311,30263,GA,2023-01-13,2023-02-13,31,kWh,Electric,Electric,10768.0,1082.74,6007,2023.01,347.354839
1,6008,2023,2,Harbor Pointe Apartments,311,30263,GA,2023-02-13,2023-03-16,31,kWh,Electric,Electric,5433.0,844.22,6008,2023.02,175.258065
2,6009,2023,3,Harbor Pointe Apartments,311,30263,GA,2023-03-16,2023-04-14,29,kWh,Electric,Electric,5052.0,826.21,6009,2023.03,174.206897
3,6010,2023,4,Harbor Pointe Apartments,311,30263,GA,2023-04-14,2023-05-14,30,kWh,Electric,Electric,3973.0,775.16,6010,2023.04,132.433333
4,6011,2023,1,Harbor Pointe Apartments,311,30263,GA,2023-01-12,2023-02-13,32,kWh,Electric,Electric,27160.0,2550.38,6011,2023.01,848.750000


In [4]:
# units for sewer should be converted to zero, otherwise consumption is counted twice
data_df.loc[data_df["UtilType"].str.contains("sewer", case=False), "Units"] = 0

In [5]:
# Group by 
grouped_df = data_df.groupby(["RYear", "RMonth", "Location","MajorType"]).agg({"Units": "sum", "Cost": "sum", "Bedrooms": "mean", "Use per Day":"sum"}).reset_index()


grouped_df.head()

,RYear,RMonth,Location,MajorType,Units,Cost,Bedrooms,Use per Day
0,2021,1,Alpine Crest Apartments,Electric,2326.000,535.34,272.0,75.032258
1,2021,1,Alpine Crest Apartments,Gas,34.900,245.59,272.0,1.057576
2,2021,1,Alpine Crest Apartments,Water,671.000,6991.92,272.0,42.030425
3,2021,1,Alpine Crest Heights,Electric,7590.000,842.51,370.0,230.000000
4,2021,1,Alpine Crest Heights,Gas,33.372,345.12,370.0,1.191857


In [6]:
grouped_df = grouped_df.loc[grouped_df["Units"] != 0]
grouped_df.head()

,RYear,RMonth,Location,MajorType,Units,Cost,Bedrooms,Use per Day
0,2021,1,Alpine Crest Apartments,Electric,2326.000,535.34,272.0,75.032258
1,2021,1,Alpine Crest Apartments,Gas,34.900,245.59,272.0,1.057576
2,2021,1,Alpine Crest Apartments,Water,671.000,6991.92,272.0,42.030425
3,2021,1,Alpine Crest Heights,Electric,7590.000,842.51,370.0,230.000000
4,2021,1,Alpine Crest Heights,Gas,33.372,345.12,370.0,1.191857


In [7]:
grouped_df.shape[0]

12272

In [8]:
rate_df = grouped_df
rate_df["unit rate"] = grouped_df["Cost"] / grouped_df["Units"]
rate_df["Use per Bdrm per Day"] = grouped_df["Use per Day"]*1000 / grouped_df["Bedrooms"]
# rate_df["Cost per Bdrm"] = grouped_df["Cost"] / grouped_df["Bedrooms"]
# rate_df["Cost per Day"] = grouped_df["Cost"] / grouped_df["Days"]
# rate_df["Use per Day"] = grouped_df["Units"] / grouped_df["Days"]
rate_df.head(20)

,RYear,RMonth,Location,MajorType,Units,Cost,Bedrooms,Use per Day,unit rate,Use per Bdrm per Day
0,2021,1,Alpine Crest Apartments,Electric,2326.000,535.34,272.0,75.032258,0.230155,275.853890
1,2021,1,Alpine Crest Apartments,Gas,34.900,245.59,272.0,1.057576,7.036963,3.888146
2,2021,1,Alpine Crest Apartments,Water,671.000,6991.92,272.0,42.030425,10.420149,154.523622
3,2021,1,Alpine Crest Heights,Electric,7590.000,842.51,370.0,230.000000,0.111003,621.621622
4,2021,1,Alpine Crest Heights,Gas,33.372,345.12,370.0,1.191857,10.341604,3.221236
6,2021,1,Alpine Heights Apartments,Electric,11693.000,1342.41,230.0,383.683333,0.114805,1668.188406
8,2021,1,Alpine Hills Apartments,Electric,15593.000,2100.93,216.0,503.543290,0.134735,2331.218935
9,2021,1,Alpine Hills Apartments,Gas,61.500,432.36,216.0,1.808824,7.030244,8.374183
10,2021,1,Alpine Hills Apartments,Water,640.000,14555.28,216.0,47.407407,22.742625,219.478738
11,2021,1,Alpine Hills Heights,Electric,9978.000,1569.00,156.0,321.870968,0.157246,2063.275434


In [9]:
print(rate_df.dtypes)

RYear                     int64
RMonth                    int64
Location                 object
MajorType                object
Units                   float64
Cost                    float64
Bedrooms                float64
Use per Day             float64
unit rate               float64
Use per Bdrm per Day    float64
dtype: object


In [10]:
output_file = "Grouped_Project_Data.csv"
rate_df.to_csv(output_file, index=False)

In [11]:
# Save the DataFrame as a JSON file
rate_df.to_json("project_data.json", orient="records")